## Send More Money cryptarithmetic puzzle

While not often spoken about as a classic data science technique,
constraint programming can be a very useful tool in numerous scenarios.
We'll look at solving a problem using brute force and then how
constraint programming provides a very declarative style
which saves us having to worry about the implementation details.

For our purposes, we'll use a classic example of a [cryptarithmetic puzzle](https://en.wikipedia.org/wiki/Verbal_arithmetic).
Such puzzles have several words arranged as a mathematical equation.
The goal is to guess each letter where each letter represents a different digit.
By convention, the leading digit of a multi-digit number should not be zero.

For us, the puzzle is:<br>
<code>&nbsp;     S E N D</code><br>
<code>&nbsp;+    M O R E</code><br>
<code>&nbsp;=  M O N E Y</code>

### Brute force approaches

First a brute force solution in Python:

In [3]:
def solutions():
    # letters = ('s', 'e', 'n', 'd', 'm', 'o', 'r', 'y')
    all_solutions = list()
    for s in range(1, 10):
        for e in range(0, 10):
            for n in range(0, 10):
                for d in range(0, 10):
                    for m in range(1, 10):
                        for o in range(0, 10):
                            for r in range(0, 10):
                                for y in range(0, 10):
                                    if len({s, e, n, d, m, o, r, y}) == 8:
                                        send = 1000 * s + 100 * e + 10 * n + d
                                        more = 1000 * m + 100 * o + 10 * r + e
                                        money = 10000 * m + 1000 * o + 100 * n + 10 * e + y
                                        if send + more == money:
                                            all_solutions.append((send, more, money))


    return all_solutions

print(solutions())

[(9567, 1085, 10652)]


Next a brute force solution in Groovy:

In [2]:
%%groovy

for (s in 1..9)
    for (e in 0..9)
        for (n in 0..9)
            for (d in 0..9)
                for (m in 1..9)
                    for (o in 0..9)
                        for (r in 0..9)
                            for (y in 0..9)
                                if ([s, e, n, d, m, o, r, y].toSet().size() == 8) {
                                    def send = 1000 * s + 100 * e + 10 * n + d
                                    def more = 1000 * m + 100 * o + 10 * r + e
                                    def money = 10000 * m + 1000 * o + 100 * n + 10 * e + y
                                    if (send + more == money) {
                                        println "s = $s, e = $e, n = $n, d = $d"
                                        println "m = $m, o = $o, r = $r, y = $y"
                                    }
                                }
OutputCell.HIDDEN

s = 9, e = 5, n = 6, d = 7
m = 1, o = 0, r = 8, y = 2


We can use permutations with Python:

In [6]:
from itertools import permutations

def solution2():
    letters = ('s', 'e', 'n', 'd', 'm', 'o', 'r', 'y')
    digits = range(10)
    for perm in permutations(digits, len(letters)):
        sol = dict(zip(letters, perm))
        if sol['s'] == 0 or sol['m'] == 0:
            continue
        send = 1000 * sol['s'] + 100 * sol['e'] + 10 * sol['n'] + sol['d']
        more = 1000 * sol['m'] + 100 * sol['o'] + 10 * sol['r'] + sol['e']
        money = 10000 * sol['m'] + 1000 * sol['o'] + 100 * sol['n'] + 10 * sol['e'] + sol['y']
        if send + more == money:
            return send, more, money

print(solution2())

(9567, 1085, 10652)


We can use permutations with Groovy:

In [7]:
%%groovy

digits = 0..9
for (p in digits.permutations()) {
    if (p[-1] < p[-2]) continue
    def (s, e, n, d, m, o, r, y) = p
    if (s == 0 || m == 0) continue
    def send = 1000 * s + 100 * e + 10 * n + d
    def more = 1000 * m + 100 * o + 10 * r + e
    def money = 10000 * m + 1000 * o + 100 * n + 10 * e + y
    if (send + more == money) {
        println "s = $s, e = $e, n = $n, d = $d"
        println "m = $m, o = $o, r = $r, y = $y"
    }
}
OutputCell.HIDDEN

s = 9, e = 5, n = 6, d = 7
m = 1, o = 0, r = 8, y = 2


### Constraint programming

We can use the [Choco constraint programming library](http://www.choco-solver.org/) which allows us to write our solution in a very declarative style using only constraints.
The set of constraints must be satisfied in every solution.
The constraint programming engine solves by applying various constraint filtering algorithms in combination with a search mechanism.
If you have heard of [Prolog](https://en.wikipedia.org/wiki/Prolog) and back-tracking, you will have the idea.

In [8]:
%%groovy

@Grab('org.choco-solver:choco-solver:4.10.2')
import org.chocosolver.solver.Model
import org.chocosolver.solver.variables.IntVar

def model = new Model("SEND+MORE=MONEY")
def S = model.intVar("S", 1, 9)
def E = model.intVar("E", 0, 9)
def N = model.intVar("N", 0, 9)
def D = model.intVar("D", 0, 9)
def M = model.intVar("M", 1, 9)
def O = model.intVar("0", 0, 9)
def R = model.intVar("R", 0, 9)
def Y = model.intVar("Y", 0, 9)

model.allDifferent(S, E, N, D, M, O, R, Y).post()

IntVar[] ALL = [
        S, E, N, D,
        M, O, R, E,
        M, O, N, E, Y]
int[] COEFFS = [
        1000, 100, 10, 1,
        1000, 100, 10, 1,
        -10000, -1000, -100, -10, -1]
model.scalar(ALL, COEFFS, "=", 0).post()

//model.solver.findSolution()

model.solver.with {
    showStatistics()
//    showDecisions()
//    showSolutions()
    findSolution()
}

- Solution #1 found. Model[SEND+MORE=MONEY], 1 Solutions, Resolution time 0.016s, 3 Nodes (183.0 n/s), 1 Backtracks, 0 Backjumps, 1 Fails, 0 Restarts 
	S = 9 E = 5 N = 6 D = 7 M = 1 0 = 0 R = 8 Y = 2 .


Solution: S=9, E=5, N=6, D=7, M=1, 0=0, R=8, Y=2, 